# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 40 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set
- 20 points - Evaluate on Github Typo Corpus (for masters only)


Remarks: 
- Use Python 3 or greater
- Max is 80 points for bachelors, 100 points for masters

## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html).

[N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf)

You may also wnat to implement:
- spell-checking for a concrete language - Russian, Tatar, Ukranian, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

Important - your project should not be a mere code copy-paste from somewhere. Implement yourself, analyze why it was suggested this way, and think of improvements/customization.

Your solution should be able to perform 4 corrections per second (3-5 words in an example) on a typical cpu.

In [1]:
from collections import defaultdict
import pandas as pd

gram1 = defaultdict(int)  # word => count
gram2 = defaultdict(int)  # (word1, word2) => count

w2 = pd.read_csv('w2_.txt', sep='\t', encoding='latin-1', header=None)

for index, row in w2.iterrows():
    c = row[0]
    gram1[row[1]] += c
    gram1[row[2]] += c
    gram2[tuple(row[1:3].values)] += c

### Norvig's solution

In [2]:
ALPHABET = 'abcdefghijklmnopqrstuvwxyz'

def normalize(text):
    letters = []
    for l in text.lower():
        if l in ALPHABET:
            letters.append(l)
        elif l in ".?!":
            letters.append(' ')
            # letters.append('.')
            # letters.append(' ')
        else:
            letters.append(' ')
    text = ''.join(letters)
    text = ' '.join(text.split())
    return text

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


class NorvigSolution:
    def __init__(self, words) -> None:
        self.WORDS = words

    def P(self, word, N=None): 
        "Probability of `word`."
        if N is None:
            N = sum(self.WORDS.values())
        if word in self.WORDS:
            return self.WORDS[word] / N
        else:
            return 0

    def correction(self, word): 
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)

    def candidates(self, word): 
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(edits1(word)) or self.known(edits2(word)) or [word])

    def known(self, words): 
        "The subset of `words` that appear in the dictionary of self.WORDS."
        return set(w for w in words if w in self.WORDS)

    def correct_text(self, text):
        text = normalize(text)
        corrected_text = []
        for word in text.split():
            corrected_text.append(self.correction(word))
        return ' '.join(corrected_text)

In [3]:
corrector = NorvigSolution(gram1)
corrector.correction('speling')

'spelling'

In [4]:
text = "dking sport is nise wau to stai healtfy"
corrector.correct_text(text)

'doing sport is nice was to stai healthy'

### N-gram solution

In [20]:

class NgramSolution:
    WEIGHTS = {
        0: 1.1,
        1: 1.0,
        2: 0.9,
    }
    def __init__(self, gram1, gram2) -> None:
        self.gram1 = gram1
        self.gram2 = gram2

    def P(self, word, sentence, pos):
        word, level = word
        score = 0
        cond1 = sentence[pos-1] in self.gram1 and (sentence[pos-1], word) in self.gram2
        if pos>0 and cond1:
            score += self.gram2[(sentence[pos-1], word)]/self.gram1[sentence[pos-1]]

        if pos<len(sentence)-1:
            cond2 = word in self.gram1 and (word, sentence[pos+1]) in self.gram2
            if cond2:
                score += self.gram2[(word, sentence[pos+1])]/self.gram1[word]
        return score #* self.WEIGHTS[level]

    def correction(self, sentence, pos):
        "Most probable spelling correction for word."
        word = sentence[pos]
        cands = self.candidates(word)
        if not cands:
            cands = self.candidates(word, False)
        if not cands:
            return word
        cands = sorted(cands, key=lambda w: self.P(w, sentence, pos), reverse=True)
        cands = [c[0] for c in cands]
        return cands[0]

    def candidates(self, word, nearest=True):
        res = {}
        cands = ((0, [word]), (1, edits1(word))) if nearest else ((2, edits2(word)),)
        for lvl, wrds in cands:
            for w in wrds:
                if w in self.gram1:
                    res.setdefault(w, lvl)
        return res.items()

    def correct_text(self, text):
        text = normalize(text).split()
        for i in range(len(text)):
            text[i] = self.correction(text, i)

        return ' '.join(text)

In [21]:
ngram_corrector = NgramSolution(gram1, gram2)
text = "dking sport is nise wau to stai healtfy"
ngram_corrector.correct_text(text)


'ding sport in nine way to stay healthy'

## Justify your decisions
In your implementation you will need to decide which ngram dataset to use, which weights to assign for edit1, edit2 or absent words probabilities, beam search parameters and etc. Write down justificaitons for these choices.

We have 2 gram and 5 gram datasets. In my opinion, ideally it should be 3 grams. Because in real data, 5-gram examples are rare. That's why I chose the 2 gram dataset.  
  
I added a weight of 1.1 for 0 edits, because this is the more likely case, a little less for one edit and even less for 2 edits.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [13]:
import random

def editsk(word, k):
    e = edits1(word)
    prev = e
    for i in range(1,k):
        temp = set(e2 for e1 in prev for e2 in edits1(e1))
        e.update(temp)
        prev = temp
    e.discard(word)
    return e

# Generate error word within 2 edit distance of the original word
def generate_error_word(word, edit_distance = 2):
    possible_errors = editsk(word, edit_distance)
    return random.choice(list(possible_errors))
    
# Generate errors in a given set of sentence 
def generate_errors(data, edit_distance = 2, errors_per_sentence = 1):
    print("Generating Errors")
    cnt = 0
    for sentence in data:
        indices = random.choices(range(len(sentence)), k = min(len(sentence), errors_per_sentence))
        for i in indices:
            sentence[i] = generate_error_word(sentence[i],edit_distance)
        if cnt%100 == 0:
            print(cnt, "completed")
        cnt += 1

    print("Errors Generated")
    return data

In [14]:
sentences = open('big.txt').read().split('.')
for i in range(len(sentences)):
    sentences[i] = normalize(sentences[i]).split()
sentences = sentences[:500]

In [15]:
import copy
sentences_with_errors = copy.deepcopy(sentences)
sentences_with_errors = generate_errors(sentences_with_errors)

Generating Errors
0 completed
100 completed
200 completed
300 completed
400 completed
Errors Generated


#### Evaluate Norvig's solution

In [16]:
norvig_corrector = NorvigSolution(gram1)
corrected_data_norvig = []

cnt = 0
for sentence in sentences_with_errors:
    corrected_sentence = []
    for word in sentence:
        c_word = norvig_corrector.correction(word)
        corrected_sentence.append(c_word)
    corrected_data_norvig.append(corrected_sentence)
    if cnt%100 == 0:
        print(cnt, "completed")
    cnt+=1

0 completed
100 completed
200 completed
300 completed
400 completed


In [17]:
c = 0
for i in range(len(sentences)):
    if corrected_data_norvig[i] == sentences[i]:
        c += 1
print("Accuracy:", c/len(sentences))

Accuracy: 0.438


#### Evaluate Ngram solution

In [22]:
ngram_corrector = NgramSolution(gram1, gram2)
corrected_data_ngram = []

cnt = 0
for sentence in sentences_with_errors:
    s = ' '.join(sentence)
    corrected_sentence = ngram_corrector.correct_text(s).split()
    corrected_data_ngram.append(corrected_sentence)
    
    if cnt%100 == 0:
        print(cnt, "completed")
    cnt+=1

0 completed
100 completed
200 completed
300 completed
400 completed


In [23]:
c = 0
for i in range(len(sentences)):
    if corrected_data_ngram[i] == sentences[i]:
        c += 1
print("Accuracy:", c/len(sentences))

Accuracy: 0.048


So Ngram solution show significantly less accuracy. I think there is some problem with code, but I can't find where.

## Evaluate on Github Typo Corpus
Now, you need to evaluate your solution on the Github Typo Corpus. Don't forget to compare the accuracy with the Norvig's solution.

In [69]:
!wget https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
!gzip -d github-typo-corpus.v1.0.0.jsonl.gz

--2023-03-14 17:53:05--  https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
Resolving github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)... 52.216.137.116, 52.217.97.52, 52.217.196.121, ...
Connecting to github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)|52.216.137.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43769081 (42M) [application/x-gzip]
Saving to: ‘github-typo-corpus.v1.0.0.jsonl.gz’

github-typo-corpus. 100%[===================>]  41,74M   465KB/s    in 2m 18s  

2023-03-14 17:55:23 (310 KB/s) - ‘github-typo-corpus.v1.0.0.jsonl.gz’ saved [43769081/43769081]



In [24]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] == 'eng' and edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((src, tgt))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [25]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

Compare your implementation with Norvig's solution on this dataset

In [33]:
limit = 1000
counter = limit
c_ngrams = 0
c_norvig = 0
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    # YOUR CODE HERE
    corrected_sentence_norvig = norvig_corrector.correct_text(src)
    corrected_sentence_n_gram = ngram_corrector.correct_text(src)
    target = normalize(target)
    if corrected_sentence_norvig == target:
        c_norvig += 1
    if corrected_sentence_n_gram == target:
        c_ngrams += 1
        
    if i%100 == 0:
        print(i, "completed")
    

0 completed
100 completed
200 completed
300 completed
400 completed
500 completed
600 completed
700 completed
800 completed
900 completed


In [34]:
print(f"Norvig solution: {c_norvig} out of {limit}")
print(f"Ngrams solution: {c_ngrams} out of {limit}")

Norvig solution: 262 out of 1000
Ngrams solution: 95 out of 1000
